In [ ]:
import pandas as pd
import requests
import time
from functools import reduce

indicators = {
    'GDP_USD': 'NY.GDP.MKTP.CD',
    'GDP_per_capita_USD': 'NY.GDP.PCAP.CD',
    'GDP_growth_pct': 'NY.GDP.MKTP.KD.ZG',
    'Inflation_CPI_pct': 'FP.CPI.TOTL.ZG',
    'Unemployment_pct': 'SL.UEM.TOTL.ZS',
    'Govt_debt_pct_GDP': 'GC.DOD.TOTL.GD.ZS',
    'Current_account_pct_GDP': 'BN.CAB.XOKA.GD.ZS',
    'Population_total': 'SP.POP.TOTL'
}

start_year, end_year = 1990, 2025

def fetch_indicator(code):
    url = f"https://api.worldbank.org/v2/country/all/indicator/{code}"
    params = {'date': f"{start_year}:{end_year}", 'format': 'json', 'per_page': 20000}
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()

    if len(data) < 2 or data[1] is None:
        return pd.DataFrame()

    df = pd.DataFrame(data[1])

    # Some "country" fields are nested dictionaries — extract "value"
    df["CountryName"] = df["country"].apply(lambda x: x["value"] if isinstance(x, dict) and "value" in x else None)
    df["CountryCode"] = df["countryiso3code"]
    df["Year"] = pd.to_numeric(df["date"], errors="coerce").astype("Int64")
    df = df[["CountryCode", "CountryName", "Year", "value"]]
    df.rename(columns={"value": code}, inplace=True)

    return df

dfs = []
for name, code in indicators.items():
    print(f"Fetching {name} ...")
    df_i = fetch_indicator(code)
    df_i.rename(columns={code: name}, inplace=True)
    dfs.append(df_i)
    time.sleep(1)

merged = reduce(lambda l, r: pd.merge(l, r, on=['CountryCode','CountryName','Year'], how='outer'), dfs)
merged = merged[(merged['Year'] >= start_year) & (merged['Year'] <= end_year)]
merged = merged[~merged['CountryCode'].isin(['WLD','EAS','ECS','LCN','MEA','NAC','SAS','SSF','EUU'])]

merged.to_csv("world_bank_macroeconomic_panel_1990_2025.csv", index=False)
print("✅ Saved: world_bank_macroeconomic_panel_1990_2025.csv")

Fetching GDP_USD ...
Fetching GDP_per_capita_USD ...
Fetching GDP_growth_pct ...
Fetching Inflation_CPI_pct ...
Fetching Unemployment_pct ...
Fetching Govt_debt_pct_GDP ...
Fetching Current_account_pct_GDP ...
Fetching Population_total ...
✅ Saved: world_bank_macroeconomic_panel_1990_2022.csv
